In [5]:
import os
import shutil
import stat
import time
from openai import OpenAI
from git import Repo, GitCommandError
import requests

def remove_readonly(func, path, exc_info):
    try:
        os.chmod(path, stat.S_IWRITE)
        func(path)
    except PermissionError:
        # Retry after small delay, useful if antivirus/git still releasing lock
        time.sleep(0.5)
        try:
            func(path)
        except Exception as e:
            print(f"⚠️ Skipping locked file: {path} ({e})")

# === 1. ENVIRONMENT CONFIG ===
api_key = os.getenv("OPENAI_API_KEY")
github_token = os.getenv("GITHUB_TOKEN")  
github_repo = os.getenv("GITHUB_REPO")  # e.g., "your-org/your-repo"
clone_dir =  r"C:\temp\ai-coder"       #os.getenv("GIT_CLONE_DIR", "/tmp/ai-coder")    
bot_name = "AI SQL Bot"
bot_email = "ai-bot@example.com"

assert api_key and github_token and github_repo, "Missing environment configuration."

# 2. Clean clone_dir if it exists
if os.path.exists(clone_dir):
    print(f"🧹 Removing existing repo at: {clone_dir}")
    shutil.rmtree(clone_dir, onerror=remove_readonly)

# Build HTTPS repo URL with token
repo_url = f"https://{github_token}@github.com/{github_repo}.git"

# Clone fresh
print(f"📥 Cloning fresh copy from {github_repo} to {clone_dir}")
repo = Repo.clone_from(repo_url, clone_dir)

# Set Git identity for commits
with repo.config_writer() as git_config:
    git_config.set_value("user", "name", bot_name)
    git_config.set_value("user", "email", bot_email)

# Ensure we're on main
repo.git.checkout("main")

# === 3. Find latest .sql file in HEAD ===
commit = repo.commit("HEAD")
target_path = None
sql_code = None

for blob in commit.tree.traverse():
    if blob.type == "blob" and blob.path.endswith(".sql"):
        target_path = os.path.join(clone_dir, blob.path)
        os.makedirs(os.path.dirname(target_path), exist_ok=True)
        sql_code = blob.data_stream.read().decode("utf-8")
        with open(target_path, "w", encoding="utf-8") as f:
            f.write(sql_code)
        break

if not sql_code:
    raise FileNotFoundError("No .sql file found in the repo.")

print(f"✅ Found SQL file: {target_path}")

# === 4. Call OpenAI to enhance the SQL ===
client = OpenAI(api_key=api_key)

enhancement_prompt = (
    "Here is a SQL script. add a join to DIM_PRODUCT from DIM_ACCOUNT  USING PRODUCT_ID \n"
    "- Reponse can have comments, but they need to be commented.  Also do not add any additional logic that was not asked for\n"
)

response = client.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": "You are an AI SQL engineer, assigned to do SQL enhancements."},
        {"role": "user", "content": f"{enhancement_prompt}\n\n```sql\n{sql_code}\n```"}
    ]
)

enhanced_sql = response.choices[0].message.content

# === 5. Overwrite file ===
with open(target_path, "w", encoding="utf-8") as f:
    f.write(enhanced_sql)

# === 6. Commit & push ===
repo.git.add(blob.path)
branch_name = f"ai-enhancement-{int(time.time())}"
repo.git.checkout("-b", branch_name)
repo.index.commit("Enhance SQL using OpenAI LLM")
repo.remote().push(refspec=f"{branch_name}:{branch_name}")

print(f"✅ Pushed changes to branch: {branch_name}")

# === 7. Open Pull Request (optional) ===
headers = {
    "Authorization": f"token {github_token}",
    "Accept": "application/vnd.github+json"
}

pr_data = {
    "title": "AI-enhanced SQL patch",
    "head": branch_name,
    "base": "main",
    "body": "This PR includes SQL enhancements generated by the AI coder."
}

response = requests.post(
    f"https://api.github.com/repos/{github_repo}/pulls",
    headers=headers,
    json=pr_data
)

if response.status_code == 201:
    pr_url = response.json().get("html_url")
    print(f"✅ Pull request created: {pr_url}")
else:
    print(f"❌ Failed to create PR: {response.text}")

🧹 Removing existing repo at: C:\temp\ai-coder
⚠️ Skipping locked file: C:\temp\ai-coder ([WinError 32] The process cannot access the file because it is being used by another process: 'C:\\temp\\ai-coder')
📥 Cloning fresh copy from ronpark71/ai_coding_poc to C:\temp\ai-coder
✅ Found SQL file: C:\temp\ai-coder\test.sql
✅ Pushed changes to branch: ai-enhancement-1759458912
✅ Pull request created: https://github.com/ronpark71/ai_coding_poc/pull/10


In [3]:
try:
    repo.close()
except Exception:
    pass